In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
import os
import sys
import json
import joblib
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
from pathlib import Path
from WattPredictor.entity.config_entity import EvaluationConfig
from WattPredictor.config.model_config import ModelConfigurationManager
from WattPredictor.utils.feature import feature_store_instance
from WattPredictor.utils.ts_generator import features_and_target, average_demand_last_4_weeks
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, root_mean_squared_error, r2_score
from WattPredictor.utils.helpers import create_directories, save_json
from WattPredictor.utils.exception import CustomException
from WattPredictor.utils.logging import logger

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.feature_store = feature_store_instance()

    def evaluate(self):
        try:
            df, _ = self.feature_store.get_training_data("elec_wx_features_view")
            df = df[['date', 'demand', 'sub_region_code', 'temperature_2m', 
                     'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]
            df.sort_values("date", inplace=True)

            if df.empty:
                raise CustomException("Loaded DataFrame is empty", sys)

            cutoff_date = (datetime.now() - timedelta(days=90)).strftime("%Y-%m-%d")
            train_df, test_df = df[df['date'] < cutoff_date], df[df['date'] >= cutoff_date]


            logger.info(f"train_df shape: {train_df.shape}, date range: {train_df['date'].min()} to {train_df['date'].max()}")
            logger.info(f"test_df shape: {test_df.shape}, date range: {test_df['date'].min()} to {test_df['date'].max()}")

            if test_df.empty:
                raise CustomException("Test DataFrame is empty after applying cutoff_date", sys)

            test_x, test_y = features_and_target(test_df, input_seq_len=self.config.input_seq_len, step_size=self.config.step_size)
            test_x.drop(columns=["date"], errors="ignore", inplace=True)

            # Validate dtypes
            non_numeric_cols = test_x.select_dtypes(exclude=['int64', 'float64', 'bool']).columns
            if not non_numeric_cols.empty:
                raise CustomException(f"Non-numeric columns found in test_x: {non_numeric_cols}", sys)

            # Validate expected features
            expected_features = [f'demand_previous_{i+1}_hour' for i in reversed(range(self.config.input_seq_len))] + \
                                ['temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']
            missing_features = [col for col in expected_features if col not in test_x.columns]
            if missing_features:
                logger.warning(f"Missing expected features in test_x: {missing_features}. Proceeding with available features.")

            model_registry = self.feature_store.project.get_model_registry()
            model_name = "wattpredictor_xgboost"
            models = model_registry.get_models(model_name)
            if not models:
                model_name = "wattpredictor_lightgbm"
                models = model_registry.get_models(model_name)
                if not models:
                    raise CustomException(f"No models found with names 'wattpredictor_xgboost' or 'wattpredictor_lightgbm'", sys)
            
            latest_model = max(models, key=lambda m: m.version)
            logger.info(f"Loading model: {model_name}, version: {latest_model.version}")

            model_dir = latest_model.download()
            model_path = os.path.join(model_dir, "model.joblib")
            pipeline = joblib.load(model_path)

            # Preprocess test_x for prediction
            test_x_transformed = test_x.copy()
            test_x_transformed = average_demand_last_4_weeks(test_x_transformed)

            preds = pipeline.predict(test_x_transformed)

            mse = mean_squared_error(test_y, preds)
            mae = mean_absolute_error(test_y, preds)
            mape = mean_absolute_percentage_error(test_y, preds) * 100
            rmse = root_mean_squared_error(test_y, preds)
            r2 = r2_score(test_y, preds)
            adjusted_r2 = 1 - (1 - r2) * (len(test_y) - 1) / (len(test_y) - test_x_transformed.shape[1] - 1)

            metrics = {
                "mse": mse,
                "mae": mae,
                "mape": mape,
                "rmse": rmse,
                "r2_score": r2,
                "adjusted_r2": adjusted_r2
            }

            create_directories([os.path.dirname(self.config.metrics_path)])
            save_json(self.config.metrics_path, metrics)
            logger.info(f"Saved evaluation metrics at {self.config.metrics_path}")

            fig, ax = plt.subplots(figsize=(12, 6))
            ax.plot(test_y[:100], label="Actual", color="blue")
            ax.plot(preds[:100], label="Predicted", color="red")
            ax.set_title("Predicted vs Actual (First 100 Points)")
            ax.set_xlabel("Time Step")
            ax.set_ylabel("Electricity Demand")
            ax.legend()

            create_directories([os.path.dirname(self.config.img_path)])
            fig.savefig(self.config.img_path)
            plt.close()
            logger.info(f"Saved prediction plot at {self.config.img_path}")

            self.feature_store.upload_file_safely(self.config.metrics_path, "eval/metrics.json")
            self.feature_store.upload_file_safely(self.config.img_path, "eval/pred_vs_actual.png")

            logger.info("Evaluation results uploaded to Hopsworks dataset storage")

            return metrics

        except Exception as e:
            raise CustomException(f"Model evaluation failed: {e}", sys)

In [3]:
try:
    config = ModelConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = Evaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-07-20 18:57:31,481: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-20 18:57:31,486: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-20 18:57:31,490: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-20 18:57:31,492: INFO: helpers: created directory at: artifacts]
[2025-07-20 18:57:31,493: INFO: helpers: created directory at: artifacts/evaluation]
[2025-07-20 18:57:31,501: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-20 18:57:31,504: INFO: external: Initializing external client]
[2025-07-20 18:57:31,505: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-20 18:57:34,527: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1240214
[2025-07-20 18:57:37,391: INFO: feature_store: Connected to Hopsworks Feature Store: WattPredictor]
Finished: Reading data from Hopsworks, using Hops

Generating TS features:   0%|          | 0/11 [00:00<?, ?it/s]

[2025-07-20 18:57:50,274: INFO: ts_generator: Columns for sub_region_code 9: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]
[2025-07-20 18:57:50,319: INFO: ts_generator: Columns for sub_region_code 5: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]
[2025-07-20 18:57:50,360: INFO: ts_generator: Columns for sub_region_code 4: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]


Generating TS features:  27%|██▋       | 3/11 [00:00<00:00, 22.77it/s]

[2025-07-20 18:57:50,405: INFO: ts_generator: Columns for sub_region_code 8: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]
[2025-07-20 18:57:50,449: INFO: ts_generator: Columns for sub_region_code 6: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]
[2025-07-20 18:57:50,493: INFO: ts_generator: Columns for sub_region_code 10: ['date', 'demand', 'temperature_2m', 'hour', 'day_of_week', 'month', 'is_weekend', 'is_holiday']]


...

[2025-07-20 18:58:01,264: INFO: helpers: created directory at: artifacts\evaluation]
[2025-07-20 18:58:01,267: INFO: helpers: json file saved at: artifacts\evaluation\metrics.json]
[2025-07-20 18:58:01,268: INFO: 2497270355: Saved evaluation metrics at artifacts\evaluation\metrics.json]
[2025-07-20 18:58:01,657: WARNING: warnings: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
]
[2025-07-20 18:58:01,690: INFO: helpers: created directory at: artifacts\evaluation]
[2025-07-20 18:58:01,847: INFO: 2497270355: Saved prediction plot at artifacts\evaluation\pred_vs_actual.png]


Uploading f:\WattPredictor\artifacts\evaluation\metrics.json: 0.000%|          | 0/210 elapsed<00:00 remaining…

[2025-07-20 18:58:04,749: INFO: feature_store: Uploaded file to Feature Store: eval/metrics.json]


Uploading f:\WattPredictor\artifacts\evaluation\pred_vs_actual.png: 0.000%|          | 0/52961 elapsed<00:00 r…

[2025-07-20 18:58:08,112: INFO: feature_store: Uploaded file to Feature Store: eval/pred_vs_actual.png]
[2025-07-20 18:58:08,115: INFO: 2497270355: Evaluation results uploaded to Hopsworks dataset storage]
